# Data Cleaning Part 2: Handle Missing Data

In [21]:
# import necessary libraries
import pandas as pd
import numpy as np
import os
# pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### Import dataset produced from first cleaning notebook

In [35]:
df = pd.read_csv('../datasets/cleaned_data.csv',index_col='Unnamed: 0')

# retain "all ages" rows and observations that dont have age data, then drop un needed features
df = pd.concat([df[df.AGEGRP=='All Ages'],df[df.AGEGRP.isnull()]]).drop(['lat', 'lng','AGEGRP',
                                'Number Food Insecure Individuals','Child FI Rate'], axis=1)

In [36]:
df.shape

(85207, 32)

# Unemployment Dataset: Fix Datatypes
Unemployment data was saved as a string due to special characters. Need to remove, convert to float, then add back into main df
### remove special characters '-' and change numeric datatypes to floats

In [37]:
# Total Workforce Column
df_workf = df[['FIPS','Year','Total_workforce']]
df_workf = df_workf.dropna()
df_workf.drop(df_workf[df_workf['Total_workforce'].values == '–'].index,inplace=True)
df_workf = df_workf['Total_workforce'].astype(float)

# Employed column
df_employed = df[['FIPS','Year','Employed']]
df_employed = df_employed.dropna()
df_employed.drop(df_employed[df_employed['Employed'].values == '–'].index,inplace=True)
df_employed = df_employed['Employed'].astype(float)

# Unemployed column
df_unemployed = df[['FIPS','Year','Unemployed']]
df_unemployed = df_unemployed.dropna()
df_unemployed.drop(df_unemployed[df_unemployed['Unemployed'].values == '–'].index,inplace=True)
df_unemployed = df_unemployed['Unemployed'].astype(float)

# Unemployment Rate column
df_unemploy_rate = df[['FIPS','Year','Unemployment_rate']]
df_unemploy_rate = df_unemploy_rate.dropna()
df_unemploy_rate.drop(df_unemploy_rate[df_unemploy_rate['Unemployment_rate'].values == '–'].index,inplace=True)
df_unemploy_rate = df_unemploy_rate['Unemployment_rate'].astype(float)


### Drop old version from main df, and add float versions back in

In [38]:
df = df.drop(['Total_workforce','Employed','Unemployed','Unemployment_rate'],axis=1)
df = pd.concat([df,df_workf,df_employed,df_unemployed,df_unemploy_rate],axis=1)

In [39]:
df

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,FIPS_state,FIPS_county,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,State/County,FI Rate,Low Threshold Type,High Threshold Type,Weighted Annual Dollars,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
0,1073,1019.995960,2014,AL-500,0.001396,0.000961,0.000435,1.0,73.0,Alabama,Jefferson County,659972.0,312491.0,347481.0,355635.0,284082.0,2029.0,10378.0,347.0,"Jefferson County, AL",0.197,SNAP,Other Nutrition Program,67209000.0,2.93,483.0,2693.0,400.0,312131.0,292505.0,19626.0,6.3
19,1117,1229.755051,2014,AL-500,0.001396,0.000961,0.000435,1.0,117.0,Alabama,Shelby County,206280.0,100304.0,105976.0,174094.0,24247.0,805.0,4403.0,101.0,"Shelby County, AL",0.105,SNAP,Other Nutrition Program,12614000.0,3.37,1.0,743.0,2706.0,107208.0,102400.0,4808.0,4.5
38,4003,1051.250000,2014,AZ-500,0.001469,0.000826,0.000643,4.0,3.0,Arizona,Cochise County,127314.0,64661.0,62653.0,112238.0,5737.0,2165.0,2757.0,510.0,"Cochise County, AZ",0.161,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",10417000.0,2.81,800.0,72.0,340.0,50969.0,46682.0,4287.0,8.4
57,4013,1095.670228,2014,AZ-502,0.001318,0.001084,0.000235,4.0,13.0,Arizona,Maricopa County,4093648.0,2024659.0,2068989.0,3449404.0,235660.0,112383.0,172425.0,11190.0,"Maricopa County, AZ",0.158,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",319193000.0,2.90,2389.0,16857.0,6320.0,1961997.0,1848119.0,113878.0,5.8
76,4019,928.546429,2014,AZ-501,0.002170,0.001705,0.000465,4.0,19.0,Arizona,Pima County,1004229.0,494684.0,509545.0,858334.0,41043.0,42683.0,31905.0,2266.0,"Pima County, AZ",0.154,"SNAP, Other Nutrition Programs","SNAP, Other Nutrition Programs",77182000.0,2.85,3591.0,838.0,1604.0,463126.0,435183.0,27943.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950228,49999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,0.0,NaN,NaN,NaN,NaN
950229,51999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,0.0,0.0,NaN,NaN,NaN,NaN
950230,53999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,0.0,0.0,NaN,NaN,NaN,NaN
950231,54999,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,0.0,0.0,NaN,NaN,NaN,NaN


In [40]:
df.dtypes

FIPS                         int64
Rent                       float64
Year                         int64
coc_number                  object
Houseless_rate             float64
Sheltered_rate             float64
Unsheltered_rate           float64
FIPS_state                 float64
FIPS_county                float64
State                       object
County                      object
TOT_POP                    float64
TOT_MALE                   float64
TOT_FEMALE                 float64
TOT_WHITE                  float64
TOT_BLACK                  float64
TOT_NATIVE                 float64
TOT_ASIAN                  float64
TOT_PACIFIC                float64
State/County                object
FI Rate                    float64
Low Threshold Type          object
High Threshold Type         object
Weighted Annual Dollars    float64
Cost Per Meal              float64
Num_wholesale              float64
Num_restaraunts            float64
Num_grocery                float64
Total_workforce     

# Feeding America Dataset: Impute Missing Values
The Feeding America datasets on food insecurity are missing county-level data for the years 2011, 2012, and 2013. I will be using data from 2010 and 2014 to calculate average yearly change, and fill in the missing values.
### Isolate rows from main df with missing values for Feeding America data

In [41]:
# create separate dataframe for each year, including features that need to be added
fi_10 = df[df.Year==2010][['FIPS','FI Rate','Cost Per Meal','Weighted Annual Dollars']]
fi_11 = df[df.Year==2011][['FIPS','FI Rate','Cost Per Meal','Weighted Annual Dollars']]
fi_12 = df[df.Year==2012][['FIPS','FI Rate','Cost Per Meal','Weighted Annual Dollars']]
fi_13 = df[df.Year==2013][['FIPS','FI Rate','Cost Per Meal','Weighted Annual Dollars']]
fi_14 = df[df.Year==2014][['FIPS','FI Rate','Cost Per Meal','Weighted Annual Dollars']]
fi_13

,FIPS,FI Rate,Cost Per Meal,Weighted Annual Dollars
364603,2020,NaN,NaN,NaN
364622,2013,NaN,NaN,NaN
364641,2016,NaN,NaN,NaN
364660,2050,NaN,NaN,NaN
364679,2060,NaN,NaN,NaN
...,...,...,...,...
950078,27999,NaN,NaN,NaN
950079,29999,NaN,NaN,NaN
950080,2270,NaN,NaN,NaN
950081,16999,NaN,NaN,NaN


### Merge together 2010 and 2014 dataframes

In [42]:
# merge and rename columns for interpretability
fi_10_and_14 = fi_10.merge(fi_14, on='FIPS',how='inner')
fi_10_and_14.rename(columns={'FI Rate_x':'FI Rate 10', 'Cost Per Meal_x': 'Meal cost 10', 
                             'FI Rate_y':'FI Rate 14', 'Cost Per Meal_y': 'Meal cost 14',
                            'Weighted Annual Dollars_x':'Dollars 10','Weighted Annual Dollars_y':'Dollars 14'}, inplace=True)
fi_10_and_14

,FIPS,FI Rate 10,Meal cost 10,Dollars 10,FI Rate 14,Meal cost 14,Dollars 14
0,2020,0.123,2.64,15945680.0,0.127,3.30,22075000.0
1,2020,0.123,2.64,15945680.0,0.127,3.30,22075000.0
2,2020,0.123,2.64,15945680.0,0.127,3.30,22075000.0
3,2013,0.145,2.69,250040.0,0.143,4.13,343000.0
4,2013,0.145,2.69,250040.0,0.143,4.13,343000.0
...,...,...,...,...,...,...,...
10165,25999,NaN,NaN,NaN,NaN,NaN,NaN
10166,26999,NaN,NaN,NaN,NaN,NaN,NaN
10167,2999,NaN,NaN,NaN,NaN,NaN,NaN
10168,4999,NaN,NaN,NaN,NaN,NaN,NaN


### Impute missing values for each year
Find difference between 2014 and 2010, and add 1/4 of the difference consecutively for each year with missing data

In [43]:
# calculate difference between 2014 and 2010 for each feature of interest
fi_10_and_14['Rate_diff'] = fi_10_and_14['FI Rate 14'] - fi_10_and_14['FI Rate 10']
fi_10_and_14['Meal_cost_dif'] = fi_10_and_14['Meal cost 14'] - fi_10_and_14['Meal cost 10']
fi_10_and_14['Dollar_diff'] = fi_10_and_14['Dollars 14'] - fi_10_and_14['Dollars 10']

# fill in the missing values for each feature and each of the three years
fi_10_and_14['FI Rate 11'] = fi_10_and_14['FI Rate 10'] + fi_10_and_14['Rate_diff']/4
fi_10_and_14['FI Rate 12'] = fi_10_and_14['FI Rate 11'] + fi_10_and_14['Rate_diff']/4
fi_10_and_14['FI Rate 13'] = fi_10_and_14['FI Rate 12'] + fi_10_and_14['Rate_diff']/4

fi_10_and_14['Meal cost 11'] = fi_10_and_14['Meal cost 10'] + fi_10_and_14['Meal_cost_dif']/4
fi_10_and_14['Meal cost 12'] = fi_10_and_14['Meal cost 11'] + fi_10_and_14['Meal_cost_dif']/4
fi_10_and_14['Meal cost 13'] = fi_10_and_14['Meal cost 12'] + fi_10_and_14['Meal_cost_dif']/4

fi_10_and_14['Dollars 11'] = fi_10_and_14['Dollars 10'] + fi_10_and_14['Dollar_diff']/4
fi_10_and_14['Dollars 12'] = fi_10_and_14['Dollars 10'] + fi_10_and_14['Dollar_diff']/4
fi_10_and_14['Dollars 13'] = fi_10_and_14['Dollars 10'] + fi_10_and_14['Dollar_diff']/4

fi_10_and_14.drop_duplicates(inplace=True)


### Create new df for each year with imputed values

In [44]:
df_11 = df[df.Year==2011]
df_11 = df_11.merge(fi_10_and_14[['FIPS','FI Rate 11','Meal cost 11','Dollars 11']], on='FIPS')
df_11.drop(['FI Rate','Weighted Annual Dollars','Cost Per Meal'],axis=1,inplace=True)
df_11.rename(columns={'FI Rate 11':'FI Rate','Meal cost 11':'Cost Per Meal','Dollars 11':'Weighted Annual Dollars'},inplace=True)


In [45]:
df_12 = df[df.Year==2012]
df_12 = df_12.merge(fi_10_and_14[['FIPS','FI Rate 12','Meal cost 12','Dollars 12']], on='FIPS')
df_12.drop(['FI Rate','Weighted Annual Dollars','Cost Per Meal'],axis=1,inplace=True)
df_12.rename(columns={'FI Rate 12':'FI Rate','Meal cost 12':'Cost Per Meal','Dollars 12':'Weighted Annual Dollars'},inplace=True)


In [46]:
df_13 = df[df.Year==2013]
df_13 = df_13.merge(fi_10_and_14[['FIPS','FI Rate 13','Meal cost 13','Dollars 13']], on='FIPS')
df_13.drop(['FI Rate','Weighted Annual Dollars','Cost Per Meal'],axis=1,inplace=True)
df_13.rename(columns={'FI Rate 13':'FI Rate','Meal cost 13':'Cost Per Meal','Dollars 13':'Weighted Annual Dollars'},inplace=True)


### Concatenate new dfs back with main df

In [47]:
# drop older missing data from main df
df_trimmed = df.drop(df[df.Year.isin([2011, 2012,2013])].index)

# add new data
df = pd.concat([df_trimmed, df_11, df_12, df_13])

### Duplicate rows?

In [71]:
df[df.Year==2010].FIPS.value_counts()

25017    12
6037     12
17197    12
17031    12
25005     9
         ..
72137     1
72153     1
72143     1
72151     1
72093     1
Name: FIPS, Length: 3245, dtype: int64

In [58]:
df[df.Year==2009]

,FIPS,Rent,Year,coc_number,Houseless_rate,Sheltered_rate,Unsheltered_rate,FIPS_state,FIPS_county,State,County,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,State/County,FI Rate,Low Threshold Type,High Threshold Type,Weighted Annual Dollars,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
916515,1001,NaN,2009,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga County, AL",0.143,SNAP,other nutrition pgm,3085290.0,2.60096,71.0,13.0,109.0,24703.0,22301.0,2402.0,9.7
916516,1003,NaN,2009,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baldwin County, AL",0.142,SNAP,other nutrition pgm,10884090.0,2.66446,0.0,588.0,627.0,82451.0,74403.0,8048.0,9.8
916517,1005,NaN,2009,NaN,NaN,NaN,NaN,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Barbour County, AL",0.236,SNAP,other nutrition pgm,3245490.0,2.77114,5.0,7.0,109.0,10003.0,8572.0,1431.0,14.3
916518,1007,NaN,2009,NaN,NaN,NaN,NaN,1.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bibb County, AL",0.178,SNAP,other nutrition pgm,1658350.0,2.59842,3.0,21.0,92.0,8742.0,7581.0,1161.0,13.3
916519,1009,NaN,2009,NaN,NaN,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Blount County, AL",0.137,SNAP,other nutrition pgm,3305260.0,2.55270,19.0,32.0,186.0,26480.0,23832.0,2648.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949991,17999,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.0,122.0,NaN,NaN,NaN,NaN
949992,18999,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.0,0.0,NaN,NaN,NaN,NaN
949993,23999,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.0,0.0,NaN,NaN,NaN,NaN
949994,6999,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,418.0,NaN,NaN,NaN,NaN


In [60]:
df[df.Year==2010].groupby('FIPS').mean()

,Rent,Year,Houseless_rate,Sheltered_rate,Unsheltered_rate,FIPS_state,FIPS_county,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,FI Rate,Weighted Annual Dollars,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,
1001,NaN,2010,0.001151,0.000762,0.000389,1.0,1.0,54647.000000,26608.333333,28038.666667,43343.333333,9712.666667,255.666667,488.333333,46.666667,0.134,3170830.0,2.58,48.0,22.0,181.0,25713.0,23431.0,2282.0,8.9
1003,NaN,2010,0.001482,0.000809,0.000673,1.0,3.0,182547.333333,89337.333333,93210.000000,159964.666667,17294.000000,1336.666667,1373.333333,123.333333,0.134,10710730.0,2.64,0.0,770.0,484.0,83459.0,75120.0,8339.0,10.0
1005,NaN,2010,0.000454,0.000382,0.000071,1.0,5.0,27413.000000,14552.000000,12861.000000,13957.000000,12925.000000,163.666667,114.000000,46.666667,0.232,2804540.0,2.53,6.0,38.0,38.0,10221.0,8959.0,1262.0,12.3
1007,NaN,2010,0.000454,0.000382,0.000071,1.0,7.0,22900.000000,12288.666667,10611.333333,17540.333333,5054.000000,67.666667,22.000000,32.333333,0.157,1558200.0,2.55,0.0,80.0,34.0,8934.0,7914.0,1020.0,11.4
1009,NaN,2010,0.000454,0.000382,0.000071,1.0,9.0,57340.000000,28369.333333,28970.666667,55382.000000,816.333333,351.000000,127.000000,62.000000,0.126,3081120.0,2.50,4.0,27.0,302.0,24906.0,22460.0,2446.0,9.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72145,NaN,2010,NaN,NaN,NaN,72.0,145.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16228.0,12856.0,3372.0,20.8
72147,NaN,2010,NaN,NaN,NaN,72.0,147.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3401.0,2767.0,634.0,18.6
72149,NaN,2010,NaN,NaN,NaN,72.0,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8841.0,6892.0,1949.0,22.0


,Rent,Year,Houseless_rate,Sheltered_rate,Unsheltered_rate,FIPS_state,FIPS_county,TOT_POP,TOT_MALE,TOT_FEMALE,TOT_WHITE,TOT_BLACK,TOT_NATIVE,TOT_ASIAN,TOT_PACIFIC,FI Rate,Weighted Annual Dollars,Cost Per Meal,Num_wholesale,Num_restaraunts,Num_grocery,Total_workforce,Employed,Unemployed,Unemployment_rate,FIPS,coc_number,State,County,State/County,Low Threshold Type,High Threshold Type
1001,NaN,2010,0.001151,0.000762,0.000389,1.0,1.0,54647.000000,26608.333333,28038.666667,43343.333333,9712.666667,255.666667,488.333333,46.666667,0.134,3170830.0,2.58,48.0,22.0,181.0,25713.0,23431.0,2282.0,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,NaN,2010,0.001482,0.000809,0.000673,1.0,3.0,182547.333333,89337.333333,93210.000000,159964.666667,17294.000000,1336.666667,1373.333333,123.333333,0.134,10710730.0,2.64,0.0,770.0,484.0,83459.0,75120.0,8339.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1005,NaN,2010,0.000454,0.000382,0.000071,1.0,5.0,27413.000000,14552.000000,12861.000000,13957.000000,12925.000000,163.666667,114.000000,46.666667,0.232,2804540.0,2.53,6.0,38.0,38.0,10221.0,8959.0,1262.0,12.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,2010,0.000454,0.000382,0.000071,1.0,7.0,22900.000000,12288.666667,10611.333333,17540.333333,5054.000000,67.666667,22.000000,32.333333,0.157,1558200.0,2.55,0.0,80.0,34.0,8934.0,7914.0,1020.0,11.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1009,NaN,2010,0.000454,0.000382,0.000071,1.0,9.0,57340.000000,28369.333333,28970.666667,55382.000000,816.333333,351.000000,127.000000,62.000000,0.126,3081120.0,2.50,4.0,27.0,302.0,24906.0,22460.0,2446.0,9.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949991,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.0,122.0,NaN,NaN,NaN,NaN,17999.0,NaN,NaN,NaN,NaN,NaN,NaN
949992,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.0,0.0,NaN,NaN,NaN,NaN,18999.0,NaN,NaN,NaN,NaN,NaN,NaN
949993,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,54.0,0.0,NaN,NaN,NaN,NaN,23999.0,NaN,NaN,NaN,NaN,NaN,NaN
949994,NaN,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,418.0,NaN,NaN,NaN,NaN,6999.0,NaN,NaN,NaN,NaN,NaN,NaN
